## Anomaly Detection

Since this is unsupervised machine learning, we don't have labels available. <br  />
So the task of a neural network is to take X and adjust the ways that it reconstructs Y,<br  />
but we don't have Y. <br  />
So hopefully, you remember what the outer encoder does. <br  />
So an outer encoder is using X as input and output as well. <br  />
So an outer encoder will try to reconstruct what it sees on the left-hand side on the right-hand side. <br  />
And if you now train the outer encoder, with an LSTM and healthy data, it tries to reconstruct the healthy data. <br  />
And remember that it runs through a neural bottleneck.<br  />
And once it's trained, it will have a hard time to reconstruct faulty data, that's the point.<br  />
That's how the anomaly detector works.

### * An LSTM Auto-encoder based Anomaly Detector in Keras
안된다 저 페이지에 안 들어가짐

In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sio
import os

In [ ]:
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/99.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/100.mat

In [ ]:
!mkdir  cwr_healthy

In [ ]:
!mv *.mat cwr_healthy/

In [ ]:
def read_folder(folder):
    data = 'dummy'
    skip = False
    for file in os.listdir(folder):
        file_id = file[:-4]
        mat_file_dict = sio.loadmat(folder+file)
        del data
        for key, value in mat_file_dict.items():
            if 'DE_time' in key or 'FE_time' in key:
                a = np.array(mat_file_dict[key])
                try:
                    data
                except NameError:
                    data = a
                else:
                    if (data.shape[0] != a.shape[0]):
                        print 'skipping ' + file_id
                        skip = True
                        continue
                    data = np.hstack((data,a))
        if skip:
            skip=False
            continue
        id = np.repeat(file_id,data.shape[0])
        id.shape = (id.shape[0],1)
        data = np.hstack((id,data))
        if data.shape[1] == 2:
            zeros = np.repeat(float(0),data.shape[0])
            zeros.shape =(data.shape[0],1)
            data = np.hstack((data,zeros))
        try:
            result
        except NameError:
            result = data
        else:
            result = np.vstack((result,data))
    return result

In [ ]:
result_healthy = read_folder('./cwr_healthy/')

In [ ]:

pdf = pd.DataFrame(result_healthy)

In [ ]:
pdf.to_csv('result_healthy_pandas.csv', header=False, index=True)

In [ ]:
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/48k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-fan-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
!mkdir cwr_faulty
!mv *.mat cwr_faulty/

In [ ]:
result_faulty = read_folder('./cwr_faulty/')

In [ ]:
pdf = pd.DataFrame(result_faulty)

In [ ]:
pdf.to_csv('result_faulty_pandas.csv', header=False, index=True)

In [ ]:
df_healhty = spark.read.csv('result_healthy_pandas.csv')
df_healhty.createOrReplaceTempView('df_healhty')

In [ ]:
spark.sql('select _c1,count(_c1) as cn from df_healhty group by _c1 order by cn asc').show()

In [ ]:
df_faulty = spark.read.csv('result_faulty_pandas.csv')
df_faulty.createOrReplaceTempView('df_faulty')

In [ ]:
spark.sql('select _c1,count(_c1) as cn from df_faulty group by _c1 order by cn asc').show()

In [ ]:
import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '6aaf54352357483486ee2d4981f8ef15',
    'region': 'dallas',
    'user_id': 'e8a574f22ee84d29b7a1987b6103fced',
    'username': 'member_adcb54bd899a7e39e31582bccad1577f68f1992f',
    'password': 'P*/m8,!#7s6H9poz'
}

configuration_name = 'os_d3bd5b94a9334de59a55a7fed2bedeaa_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
df_healhty.write.parquet(bmos.url('courseraai', 'cwr_healthy1.parquet'))

In [ ]:
df_faulty.write.parquet(bmos.url('courseraai', 'cwr_faulty1.parquet'))

In [ ]:
df_faulty.count()

## Keras에서 돌리기

In [16]:
import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '6aaf54352357483486ee2d4981f8ef15',
    'region': 'dallas',
    'user_id': 'e8a574f22ee84d29b7a1987b6103fced',
    'username': 'member_adcb54bd899a7e39e31582bccad1577f68f1992f',
    'password': 'P*/m8,!#7s6H9poz'
}

configuration_name = 'os_d3bd5b94a9334de59a55a7fed2bedeaa_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [17]:
df_healthy = spark.read.parquet(bmos.url('courseraai', 'cwr_healthy.parquet'))
df_healthy.createOrReplaceTempView('df_healthy')

In [18]:
df_faulty = spark.read.parquet(bmos.url('courseraai', 'cwr_faulty.parquet'))
df_faulty.createOrReplaceTempView('df_faulty')

In [19]:
df_healthy = df_healthy.repartition(1)

In [21]:
df_healthy.write.csv('cwr_healthy2.csv')

In [22]:
df_faulty = df_faulty.repartition(1)

In [24]:
df_faulty.write.csv('cwr_faulty2.csv')

In [28]:
import pandas as pd
df_healthy = pd.read_csv('cwr_healthy2.csv/part-00000-c08ced3c-6ac7-4da9-be74-33077e7844b5.csv', engine='python', header=None)
df_healthy.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cwr_healthy2.csv/part-00000-c08ced3c-6ac7-4da9-be74-33077e7844b5.csv'

In [29]:
df_healthy.loc[df_healthy[1] == 100]

AttributeError: 'DataFrame' object has no attribute 'loc'

In [30]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib inline

Using TensorFlow backend.


In [31]:
def get_recording(df,file_id):
    return np.array(df.orderBy(df['_c0']).where(df['_c1'] == file_id).select('_c2','_c3').rdd.map(lambda row: np.array([row._c2,row._c3])).collect())

In [32]:
def get_recording(df,file_id):
    return np.array(df.sort_values(by=0, ascending=True).loc[df[1] == file_id].drop(0,1).drop(1,1))

In [33]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_d3bd5b94a9334de59a55a7fed2bedeaa(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_adcb54bd899a7e39e31582bccad1577f68f1992f','domain': {'id': '4619da2fa8524beda11c89d2d1969c5b'},
            'password': 'P*/m8,!#7s6H9poz'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_d3bd5b94a9334de59a55a7fed2bedeaa('courseraai', 'cwr_healthy.csv/part-00000-2fe5401d-6223-4931-880e-634da433b039.csv-attempt_20180202060424_0005_m_000000_0'))
df_data_1.head()

,<html><h1>Not Found</h1><p>The resource could not be found.</p></html>


In [34]:
import numpy as np
healthy_sample = get_recording(df_healthy,100)
faulty_sample = get_recording(df_faulty,125)

AttributeError: 'DataFrame' object has no attribute 'sort_values'

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(healthy_sample)
ax.plot(range(0,size), healthy_sample[:,0], '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,size), healthy_sample[:,1], '-', color='blue', animated = True, linewidth=1)

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(faulty_sample)
ax.plot(range(0,size), faulty_sample[:,1], '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,size), faulty_sample[:,0], '-', color='blue', animated = True, linewidth=1)

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
ax.plot(range(0,500), healthy_sample[:500,0], '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,500), healthy_sample[:500,1], '-', color='blue', animated = True, linewidth=1)

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
ax.plot(range(0,500), faulty_sample[:500,0], '-', color='red', animated = True, linewidth=1)
ax.plot(range(0,500), faulty_sample[:500,1], '-', color='blue', animated = True, linewidth=1)

In [ ]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
timesteps = 100
dim = 2
lossHistory = LossHistory()
# design network

model = Sequential()
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(Dense(2))
model.compile(loss='mae', optimizer='adam')

def train(data):
    model.fit(data, data, epochs=20, batch_size=72, validation_data=(data, data), verbose=1, shuffle=False,callbacks=[lossHistory])

def score(data):
    yhat =  model.predict(data)
    return yhat

In [ ]:
def create_trimmed_recording(df,file_id):
    recording = np.array(df.orderBy(df['_c0']).where(df['_c1'] == file_id).select('_c2','_c3').rdd.map(lambda row: np.array([row._c2,row._c3])).collect())  
    samples = len(recording)
    trim = samples % 100
    recording_trimmed = recording[:samples-trim]
    recording_trimmed.shape = (samples/timesteps,timesteps,dim)
    return recording_trimmed

In [ ]:
def create_trimmed_recording(df,file_id):
    recording = get_recording(df,file_id) 
    samples = len(recording)
    trim = samples % 100
    recording_trimmed = recording[:samples-trim]
    recording_trimmed.shape = (samples/timesteps,timesteps,dim)
    return recording_trimmed

In [ ]:
#pd.unique()
#df_healthy.drop(0,1).drop(2,1).drop(3,1)
pd.unique(df_healthy.iloc[:,1])

In [ ]:
file_ids = pd.unique(df_healthy.iloc[:,1])
start = time.time()
for file_id in file_ids:
    recording_trimmed = create_trimmed_recording(df_healthy,file_id)
    print "Staring training on %s" % (file_id)
    train(recording_trimmed)
    print "Finished training on %s after %s seconds" % (file_id,time.time()-start)

print "Finished job on after %s seconds" % (time.time()-start)
healthy_losses = lossHistory.losses

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(healthy_losses)
plt.ylim(0,0.008)
ax.plot(range(0,size), healthy_losses, '-', color='blue', animated = True, linewidth=1)

In [ ]:
#file_ids = spark.sql('select distinct _c1 from df_healhty').rdd.map(lambda row : row._c1).collect()
start = time.time()
for file_id in [105]:
    recording_trimmed = create_trimmed_recording(df_faulty,file_id)
    print "Staring training on %s" % (file_id)
    train(recording_trimmed)
    print "Finished training on %s after %s seconds" % (file_id,time.time()-start)

print "Finished job on after %s seconds" % (time.time()-start)
faulty_losses = lossHistory.losses

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(healthy_losses+faulty_losses)
plt.ylim(0,0.008)
ax.plot(range(0,size), healthy_losses+faulty_losses, '-', color='blue', animated = True, linewidth=1)